In [1]:
import sys
sys.path.append("..")

In [2]:
from solvers.MMR.PQK_solver import *
from circuits.circuits import *
from utils.rbf_kernel_tools import *
from squlearn.encoding_circuit import *
from squlearn.kernel import *

We will show that the kernel derivatives calculated by squlearn are the same as the ones analytically calculated for the separable rx fmap

In [3]:
from circuits.circuits import Separable_rx

Same K(X, X) 1D

In [4]:
import sympy as sp

x,y,gamma_sp = sp.symbols("x y gamma")

#create x_vec a tensor with 1, 2 , 3
x_vec = sp.IndexedBase("x")
y_vec = sp.IndexedBase("y")
 
def k_(x_vec, y_vec, n):
    s = 0
    for i in range(0, n):
        s+= 2 - 2*sp.cos(x_vec[i]-y_vec[i])
    total = sp.exp(-gamma_sp*s)
    return total

def substitute_manually(obj, x, y, gamma_num):
    dictionary_to_sub = {}
    dictionary_to_sub[gamma_sp] = gamma_num
    for i in range(len(x)):
        dictionary_to_sub[x_vec[i]] = x[i]
        dictionary_to_sub[y_vec[i]] = y[i]
    return obj.subs(dictionary_to_sub).evalf()
    

    

In [14]:
x_train = np.array([[0, 1],[1, .5]])
y_train = np.array([[1],[0]])
x_train = np.array([[0],[1]])

n_feature = x_train.shape[1]
feature_map = Separable_rx(n_feature,1,n_feature)
q_kernel = ProjectedQuantumKernel(feature_map, Executor())

In [18]:
def eval_k_matrix(x_dataset, y_dataset):
    num_matrix = np.zeros((len(x_dataset), len(y_dataset)))
    num_dimensions = len(x_dataset[0])
    for i in range(len(x_dataset)):
        for j in range(len(y_dataset)):
            num_matrix[i,j] = substitute_manually(k_(x_vec, y_vec, num_dimensions), x_dataset[i], y_dataset[j], 1)
    return num_matrix

def eval_k_matrix_dx(x_dataset, y_dataset):
    num_matrix = np.zeros((len(x_dataset), len(y_dataset), len(x_dataset[0])))
    num_dimensions = len(x_dataset[0])
    for i in range(len(x_dataset)):
        for j in range(len(y_dataset)):
            for l in range(len(x_dataset[0])):
                num_matrix[i,j, l] = substitute_manually(k_(x_vec, y_vec, num_dimensions).diff(x_vec[l]), x_dataset[i], y_dataset[j], 1)
    return num_matrix

eval_k_matrix(x_train, y_train)

array([[0.39876006, 1.        ],
       [1.        , 0.39876006]])

In [19]:
q_kernel.evaluate(x_train, y_train)

Calculating f
Calculating f


array([[0.39876006, 1.        ],
       [1.        , 0.39876006]])

In [22]:
q_kernel.evaluate_derivatives(x_train, y_train, "dfdx")

Calculating dfdx
Calculating f
Shape of x_result (O) (2, 3)
Calculating f


array([[ 0.67109005,  0.        ],
       [ 0.        , -0.67109005]])

In [21]:
eval_k_matrix_dx(x_train, y_train)[:,:,0]

array([[ 0.67109005,  0.        ],
       [ 0.        , -0.67109005]])

In [ ]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')
x3, y3 = sp.symbols('x_3 y_3')
O1 = sp.symbols('O_1', cls=sp.Function)
O2 = sp.symbols('O_2', cls=sp.Function)
O3 = sp.symbols('O_3', cls=sp.Function)


In [ ]:
k = sp.symbols('k', cls=sp.Function)
pqk_fun =  k(O1(x), O2(x), O3(x), O1(y), O2(y), O3(y))
pqk_fun_dx = sp.diff(pqk_fun, x).simplify()
pqk_fun_dxdx = sp.diff(pqk_fun_dx, x).simplify()

Fundamentally, we implement the below equation using einsum notation

In [ ]:
pqk_fun_dxdx.simplify()

Derivative(O_1(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2)) + 2*Derivative(O_1(x), x)*Derivative(O_2(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_2(x)) + 2*Derivative(O_1(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_3(x)) + Derivative(O_1(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x)) + Derivative(O_2(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_2(x), 2)) + 2*Derivative(O_2(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x), O_3(x)) + Derivative(O_2(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x)) + Derivative(O_3(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2)) + Derivative(O_3(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_3(x))

In [ ]:
from squlearn.kernel.matrix import ProjectedQuantumKernel

In [ ]:
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1), executor=Executor("pennylane"), gamma=gamma)

In [ ]:
pqk_squlearn.evaluate(X, X)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [ ]:
K_separable_rx_PQK_(X,X, gamma)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [ ]:
K_f

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [ ]:
K_separable_rx_PQK_dx(X,X,gamma)

array([[-0.       , -0.4298443],
       [ 0.4298443, -0.       ]])

In [ ]:
K_dfdx

array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [ ]:
K_separable_rx_PQK_dxdx(X,X,gamma)

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [ ]:
K_dfdxdx

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [ ]:
import numpy as np
#from sklearn import RBF
from sklearn.gaussian_process.kernels import RBF

In [ ]:
from circuits.circuits import Separable_rx

In [ ]:
sigma=1
gamma= 1/(2*sigma**2)
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1, num_features=1), executor=Executor("pennylane"), gamma=gamma)

In [ ]:
X = [0.1, -0.41]

In [ ]:
pqk_squlearn.evaluate_derivatives(X, X, "dfdx")

Calculating dfdx
Calculating f
Calculating f


array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [ ]:
pqk_squlearn.evaluate_derivatives(X, X, "dfdxdx")

Calculating dfdx
Calculating dfdxdx
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f
Calculating f


array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])